#Load Silver Layer

In [0]:
%run "./ADLS Setup Variables_SP"


In [0]:
from pyspark.sql.functions import when, from_json, col, explode, lit, udf, split, count, size, element_at, hour, day, weekofyear, month, year, date_format
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F      # can remove
import pandas as pd
import matplotlib.pyplot as plt

# Org Dim Write

In [0]:
org_dim = spark.read.format('parquet').load(silver_path + "OrgTable") # no change needed 

In [0]:
(org_dim
 .repartition(2)
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "OrgDim")
)

# Repo Dim Write

In [0]:
repo_dim = spark.read.format('parquet').load(silver_path + "RepoTable") # no change needed 

In [0]:
(repo_dim
 .repartition(4)
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "RepoDim")
)


# Actor Dim Write

In [0]:
actor_dim = spark.read.format('parquet').load(silver_path + "ActorTable") # no change needed

In [0]:
(actor_dim
 .repartition(2)
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "ActorDim")
)

# Event Dim Write


In [0]:
event_fact = spark.read.format('parquet').load(silver_path + "EventTable")

In [0]:
#create event_dim
(event_fact
 .select(col("id") ,col("event_type").alias("type"), col("public"), col("created_at"))
 .repartition(12)
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "EventDim")
)

# Event FACT Write

In [0]:
event_fact = event_fact.drop(col("event_type"), col("public"), col("created_at"))

In [0]:
pe_dim = (spark.read.format('parquet').load(silver_path + "PushEventTable")
          )

In [0]:
event_fact = (event_fact
              .join(pe_dim.drop("is_main"), event_fact.id == pe_dim.event_id, "outer")
              .drop("branch_ref")
              .withColumnRenamed("size","pe_size")
              .withColumnRenamed("distinct_size","pe_distinct_size")
              ).drop("event_id")


In [0]:
(event_fact
 .repartition(24) 
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "EventFact")
)

# Push Event Dim Write

In [0]:
pe_dim = pe_dim.select(col("push_id").alias("id"), col("branch_ref").alias("ref"), col("is_main"))

In [0]:

(pe_dim
 .repartition(4) 
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "PushEventDim")
)

# Commit Dim Write

In [0]:
#commit_dim = (spark.read.format('parquet').load(silver_path + "CommitTable")
#              .drop("distinct").withColumnRenamed("language", "commit_language")
#              )

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-802039001853887>, line 1
----> 1 commit_dim = (spark.read.format('parquet').load(silver_path + "CommitTable")
      2               .drop("distinct").withColumnRenamed("language", "commit_language")
      3               )

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(

In [0]:
"""
#create event_dim
(commit_dim
 .repartition(22) # do some math for this one...
 .write
 .format("parquet")
 .option("header", True)
 .mode("overwrite")       
 .save(gold_path + "CommitDim")
)
"""